In [ ]:
import numpy as np
import gzip
import os
import tensorflow as tf
import re

# specify the path to the zipped file.
os.chdir('/mnt/c/Users/amsj1/OneDrive - University of Tennessee/2nd_year/BZAN554_deep_learning/bzan554-group-assignment1')

In [ ]:
def parse(path):
    """
    Function to read in the VERY LARGE dataset and yield it as a generator for
    memory efficiency. Takes one argument which is the path to the file being
    read in. This path is set abose using 'os.chdir'.
    """
    with gzip.open(path, 'rb') as g:
        for l in g:
            yield eval(l)

#######################################################################
# Basically load in some data to make lookup tables for the sequentially
# learning part
#######################################################################
lookup_X = []
Y_raw = []
counter = 0
for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'):
    counter += 1
    # X_raw.append(d['title'])
    X_raw = np.array(d['title'])
    Y_raw.append(d['category'])

    X_words = [words for words in str(X_raw).lower().split()]
    lookup_X.extend(X_words)

    if counter == 1000:
        break

# flatten the Y list of lists
flat_Y = [category for subcat in Y_raw for category in subcat]
# uniquefy it
unique_Y = np.unique(np.array(flat_Y))

# look up for Y
lookup_Y = []
lookup_Y.extend([category for category in unique_Y])

In [ ]:
#######################################################################
# Set up the simpole model architecture
#######################################################################
inputs = tf.keras.layers.Input(shape = (len(lookup_X),))
hidden1 = tf.keras.layers.Dense(
    units = len(lookup_X),
    activation = 'sigmoid',
    name = 'hidden1'
)(inputs)
hidden2 = tf.keras.layers.Dense(
    units = len(lookup_Y),
    activation = 'sigmoid',
    name = 'hidden2'
)(hidden1)
output = tf.keras.layers.Dense(
    units = len(unique_Y),
    activation = 'sigmoid',
    name = 'output'
)(hidden2)

#######################################################################
# create & compile the simple model
#######################################################################
model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate = 0.001),
    metrics=[tf.keras.metrics.Accuracy()]
)

In [ ]:
#######################################################################
# Set up the complex model architecture
# NOTE: This was ran on Dan's machine... so variable names are the same
# and will be overwritten
#######################################################################
inputs = tf.keras.layers.Input(shape = (len(lookup_X),))
hidden1 = tf.keras.layers.Dense(
    units = len(lookup_X),
    activation = 'sigmoid',
    name = 'hidden1'
)(inputs)
hidden2 = tf.keras.layers.Dense(
    units = len(lookup_Y),
    activation = 'sigmoid',
    name = 'hidden2'
)(hidden1)
hidden3 = tf.keras.layers.Dense(
    units = len(lookup_Y)*2,
    activation = 'sigmoid',
    name = 'hidden3'
)(hidden2)
hidden4 = tf.keras.layers.Dense(
    units = len(lookup_Y)*2,
    activation = 'sigmoid',
    name = 'hidden4'
)(hidden3)
hidden5 = tf.keras.layers.Dense(
    units = len(lookup_Y)*4,
    activation = 'sigmoid',
    name = 'hidden5'
)(hidden4)
hidden6 = tf.keras.layers.Dense(
    units = len(lookup_Y)*4,
    activation = 'sigmoid',
    name = 'hidden6'
)(hidden5)
hidden7 = tf.keras.layers.Dense(
    units = len(lookup_Y)*4,
    activation = 'sigmoid',
    name = 'hidden7'
)(hidden6)
output = tf.keras.layers.Dense(
    units = len(unique_Y),
    activation = 'sigmoid',
    name = 'output'
)(hidden7)

#######################################################################
# create & compile the complex model
#######################################################################
model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.001)
)

In [ ]:
#######################################################################
# Big boy model learning time
#######################################################################
for i in range(10): # number of epochs
    print(f'Epoch number: {i}')
    counter = 0
    for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'): # instances
        counter += 1
        X_raw = np.array(d['title'])
        Y_raw = np.array(d['category'])

        ###############################################
        # logic for the X hot-one-encoded stuff
        ###############################################
        # split the titles into words and lowercase
        # words = word.lower().split()
        X_words = [words for words in str(X_raw).lower().split()]

        # Get final binary stuff for X
        X_indices = np.where(np.isin(lookup_X, X_words))
        X_final = np.zeros(len(lookup_X))
        X_final[X_indices] = 1
        X_final = X_final.reshape(1, len(X_final))

        ###############################################
        # logic for the Y hot-one-encoded stuff
        ###############################################
        Y_indices = np.where(np.isin(lookup_Y, unique_Y))
        Y_final = np.zeros(len(lookup_Y))
        Y_final[Y_indices] = 1
        Y_final = Y_final.reshape(1, len(Y_final))

        ###############################################
        # Finallllyyyyy fit the freakin model
        ###############################################
        model.fit(x=X_final,y=Y_final,epochs=1,batch_size=1)

        if counter == 1000:
            break

In [ ]:
# overall predictions
yhat = model.predict(x=X_final)

# make copy and get out the indices of the top 5 probabilities
yhat_copy = yhat.copy()
result_ind = np.argpartition(yhat_copy, -5)[0][-5:]
result_ind

# Get the categories and their highest probability
np.array(unique_Y)[result_ind]
yhat_copy[0][result_ind]
cats_probs = np.column_stack(
    [yhat_copy[0][result_ind],
    np.array(unique_Y)[result_ind]]
)